In [1]:
import tensorflow as tf
import gpflow
import tensorflow_probability as tfp
import numpy as np

tf.random.set_seed(7)


2024-05-22 09:32:35.646322: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 09:32:35.840065: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 09:32:36.463423: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 09:32:38.006751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def generate_regressor(x_original, n_regressors):
    """Pad X with zeros from left for #regressors-1 for each data point
    roll over X where window size equals #regressors

    Args:
        x_original (_type_): _description_
        n_regressors (_type_): _description_

    Returns:
        _type_: _description_
    """
    X = []
    x_padded = np.pad(x_original, pad_width=(n_regressors, 0), mode="constant", constant_values=(0))
    for _ in range(len(x_original)):
        x_padded = np.roll(x_padded, -1)
        X.append(x_padded[:n_regressors])

    X = np.array(X, dtype='f')

    return X


def load_data(train_in_path, train_out_path, test_in_path, test_out_path, size_train: int, size_test: int, n_regressors: int):
    """_summary_

    Args:
        train_in_path (_type_): _description_
        train_out_path (_type_): _description_
        test_in_path (_type_): _description_
        test_out_path (_type_): _description_
        size (int): _description_
        n_regressors (int): _description_

    Returns:
        _type_: _description_
    """

    x_train_in = np.loadtxt(train_in_path, dtype='f')[:size_train]
    x_test_in = np.loadtxt(test_in_path, dtype='f')[:size_test]

    X_train = generate_regressor(x_train_in, n_regressors)
    X_test = generate_regressor(x_test_in, n_regressors)

    Y_train = np.loadtxt(train_out_path, dtype='f')[:size_train, None]
    Y_test = np.loadtxt(test_out_path, dtype='f')[:size_test, None]

    return X_train, Y_train, X_test, Y_test

In [97]:
def train(X, Y, params_summary: bool = False):
    """_summary_

    Args:
        X (_type_): _description_
        Y (_type_): _description_

    Returns:
        _type_: _description_
    """
    model = gpflow.models.GPR(
        (X, Y),
        kernel= gpflow.kernels.SquaredExponential(variance=1.0, lengthscales=1.0), # gpflow.kernels.White(0.1) + 
        # mean_function = gpflow.functions.Constant(),
        likelihood = gpflow.likelihoods.Gaussian(0.1)
        # noise_variance = 0.2,
    )
    
    if params_summary:
        gpflow.utilities.print_summary(model)

    print(model.trainable_variables)
    # opt = gpflow.optimizers.Scipy()
    # opt.minimize(model.training_loss, model.trainable_variables)

    return model

In [100]:
train_in_file = "../src/data/training/training_input.txt"
train_out_file = "../src/data/training/training_output.txt"
test_in_file = "../src/data/test/test_input.txt"
test_out_file = "../src/data/test/test_output.txt"

size_train = 10
size_test = 10
n_regressors = 100

X_train, Y_train, X_test, Y_test = load_data(
    train_in_file,
    train_out_file,
    test_in_file,
    test_out_file,
    size_train,
    size_test,
    n_regressors,
)

gpflow.config.set_default_summary_fmt("notebook")
gpflow.config.set_default_float(np.float32)
model = train(X_train, Y_train)

(<tf.Variable 'softplus:0' shape=() dtype=float32, numpy=0.54132485>, <tf.Variable 'softplus:0' shape=() dtype=float32, numpy=0.54132485>, <tf.Variable 'chain_of_shift_of_softplus:0' shape=() dtype=float32, numpy=-2.252179>)


In [101]:
gpflow.utilities.set_trainable(model.kernel.lengthscales, True)
gpflow.utilities.set_trainable(model.kernel.variance, False)
gpflow.utilities.set_trainable(model.likelihood, False)
    
model.trainable_variables

(<tf.Variable 'softplus:0' shape=() dtype=float32, numpy=0.54132485>,)

In [102]:
# model.trainable_parameters
param = model.kernel.parameters[0]
print(param)
print(param.unconstrained_variable)  # this is what the optimizer will see
print(param.transform)  # the above two are related via this

<Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=1.0>
<tf.Variable 'softplus:0' shape=() dtype=float32, numpy=0.54132485>
tfp.bijectors.Softplus("softplus", batch_shape=[], min_event_ndims=0)


In [103]:
# model.trainable_parameters
param = model.likelihood.parameters[0]
print(param)
print(param.unconstrained_variable)  # this is what the optimizer will see
print(param.transform)  # the above two are related via this
print(param.transform.bijectors[0].parameters)
print(param.transform.bijectors[1].parameters)

<Parameter: name=chain_of_shift_of_softplus, dtype=float32, shape=[], fn="chain_of_shift_of_softplus", numpy=0.1>
<tf.Variable 'chain_of_shift_of_softplus:0' shape=() dtype=float32, numpy=-2.252179>
tfp.bijectors.Chain("chain_of_shift_of_softplus", batch_shape=[], min_event_ndims=0, bijectors=[Shift, Softplus])
{'shift': <tf.Tensor: shape=(), dtype=float32, numpy=1e-06>, 'validate_args': False, 'name': 'shift'}
{'hinge_softness': None, 'low': None, 'validate_args': False, 'name': 'softplus'}


In [104]:
p = np.array([0.001], dtype='f')
soft = tfp.bijectors.Softplus(hinge_softness=None, validate_args=False, low=None)
sh = tfp.bijectors.Shift([1e-06], validate_args=False)
p_u = soft.inverse(sh.inverse([p]))
print(p_u)
p_f = sh.forward(soft.forward(p_u))
print(f"{p_f.numpy()[0][0]:.6f}")

tf.Tensor([[-6.908256]], shape=(1, 1), dtype=float32)
0.001000


In [105]:

def my_callback() -> None:
    print(f'{(model.training_loss()/X_train.shape[0]).numpy():.7f}', " ", model.kernel.lengthscales)#gpflow.utilities.print_summary(model))

execute_task = gpflow.monitor.ExecuteCallback(my_callback)
task_group = gpflow.monitor.MonitorTaskGroup(execute_task)
monitor = gpflow.monitor.Monitor(task_group)
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

@tf.function
def optimization_step(): 
    with tf.GradientTape() as tape:
        loss = model.training_loss()
    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))

print("inital loss ", model.training_loss()/X_train.shape[0], " ", gpflow.utilities.print_summary(model))

for i in range(10): 
    optimization_step() 
    
    param = model.kernel.parameters[0]
    # print(param)
    # print(param.unconstrained_variable)  # this is what the optimizer will see
    # print(param.transform)  # the above two are related via this
    
    # if i % 10 == 0:
    monitor(i)
    
gpflow.utilities.print_summary(model)

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,False,(),float32,1
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,1
GPR.likelihood.variance,Parameter,Softplus + Shift,,False,(),float32,0.1


inital loss  tf.Tensor(1.2938163, shape=(), dtype=float32)   None
1.2937419   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.999368>
1.2936685   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.9987364>
1.2935956   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.9981053>
1.2935236   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.99747485>
1.2934520   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.9968451>
1.2933815   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.99621624>
1.2933118   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.9955883>
1.2932428   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.99496144>
1.2931747   <Parameter: name=softplus, dtype=float32, shape=[], fn="softplus", numpy=0.9943358>
1.2931073   <Parameter: name=softplus, dtype=float32, shape=[], fn="

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,False,(),float32,1
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,0.993711
GPR.likelihood.variance,Parameter,Softplus + Shift,,False,(),float32,0.1


In [37]:
# model.trainable_parameters
param = model.likelihood.parameters[0]
print(param)
print(param.unconstrained_variable)  # this is what the optimizer will see
print(param.transform)  # the above two are related via this

<Parameter: name=chain_of_shift_of_softplus, dtype=float32, shape=[], fn="chain_of_shift_of_softplus", numpy=0.1>
<tf.Variable 'chain_of_shift_of_softplus:0' shape=() dtype=float32, numpy=-2.252179>
tfp.bijectors.Chain("chain_of_shift_of_softplus", batch_shape=[], min_event_ndims=0, bijectors=[Shift, Softplus])


In [45]:
def my_callback() -> None:
    print(model.training_loss(), " ", gpflow.utilities.print_summary(model))

execute_task = gpflow.monitor.ExecuteCallback(my_callback)
task_group = gpflow.monitor.MonitorTaskGroup(execute_task, period=3)
monitor = gpflow.monitor.Monitor(task_group)

opt = gpflow.optimizers.Scipy()

for i in range(10): 
    opt.minimize(model.training_loss, model.trainable_variables, step_callback=monitor)
    monitor(i)

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float32,0.795218
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,1.69561
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float32,0.00195985


tf.Tensor(3.9486237, shape=(), dtype=float32)   None


name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float32,0.813227
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,2.31663
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float32,0.00135743


tf.Tensor(-4.307831, shape=(), dtype=float32)   None


name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float32,1.91122
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,3.6353
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float32,0.000165338


...

tf.Tensor(-9.142197, shape=(), dtype=float32)   None


name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float32,4.693
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,5.63877
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float32,2.82107e-06


tf.Tensor(-9.142197, shape=(), dtype=float32)   None


name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.variance,Parameter,Softplus,,True,(),float32,4.693
GPR.kernel.lengthscales,Parameter,Softplus,,True,(),float32,5.63877
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float32,2.82107e-06


tf.Tensor(-9.142197, shape=(), dtype=float32)   None
